In [1]:
from src.data.data_manager import DataManager
from src.data.data_labeling import DataLabeler
from src.features.feature_engineering import BuildFeatures
from src.features.feature_selection import SelectFeatures
from hydra import initialize, compose
from omegaconf import DictConfig
import pandas as pd
import numpy as np
import polars as pl
import time
import random as rd

In [2]:
initialize(version_base=None, config_path="config", job_name="test")
cfg = compose(config_name="general.yaml", return_hydra_config=True)
dm = DataManager(cfg)
dl = DataLabeler(cfg)
build_features = BuildFeatures(cfg)
select_features = SelectFeatures(cfg)

In [3]:
sh = dm.load_share('BBG0100R9963')

In [4]:
points = []

In [5]:
timer = int(time.time())
price = 10000
for j in range(5000):
    price += (rd.random() - 0.5) * 10 * (rd.random() + rd.random() + rd.random() + rd.random() + rd.random() + rd.random() + rd.random() + rd.random() + rd.random() + rd.random() + rd.random() + rd.random() + rd.random() + rd.random() + rd.random() + rd.random() + rd.random())
    point = [1 for i in range(29)]
    point[0] = timer
    point[2] = rd.randint(0, 100)
    point[4] = rd.randint(0, 100)
    point[8] = price
    points.append(point)
    timer += rd.randint(1, 5)

In [6]:
# point = [1 for i in range(29)]
# point[0] = int(time.time())

In [7]:
sh = pl.DataFrame(points, schema=sh.columns)

In [8]:
sh, labels = dl.label_dataset(sh)

In [9]:
for i in range(3000):
    labels[rd.randint(0, 4000)] = 1 if price > 10000 else -1

In [10]:
select_features(sh.select(['price_last', 'price_min', 'buyers_quantity']), labels)

Features less important than random noise:
- price_min
- buyers_quantity
[0.71379385 0.         0.28620615] [0. 0. 0.]
